In [1]:
import os
import math
import random
import time
import numpy as np
import tensorflow as tf
import cv2
import threading
from PIL import Image

slim = tf.contrib.slim

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
import sys
sys.path.append('../')

In [4]:
from nets import ssd_vgg_300, ssd_common, np_methods
from preprocessing import ssd_vgg_preprocessing
from notebooks import visualization

In [5]:
# TensorFlow session: grow memory when needed. TF, DO NOT USE ALL MY GPU MEMORY!!!
gpu_options = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(log_device_placement=False, gpu_options=gpu_options)

# config = tf.ConfigProto(device_count={"CPU": 4}, 
#                 inter_op_parallelism_threads = 1,   
#                 intra_op_parallelism_threads = 4,  
#                 log_device_placement=False)
# # 创建队列
# # 队列有两个int32的元素
# q = tf.FIFOQueue(2,'int32')
# # 初始化队列
# init= q.enqueue_many(([0,10],))

isess = tf.InteractiveSession(config=config)

## SSD 300 Model

The SSD 300 network takes 300x300 image inputs. In order to feed any image, the latter is resize to this input shape (i.e.`Resize.WARP_RESIZE`). Note that even though it may change the ratio width / height, the SSD model performs well on resized images (and it is the default behaviour in the original Caffe implementation).

SSD anchors correspond to the default bounding boxes encoded in the network. The SSD net output provides offset on the coordinates and dimensions of these anchors.

In [23]:
# Input placeholder.
net_shape = (300, 300)
data_format = 'NHWC'
img_input = tf.placeholder(tf.uint8, shape=(300, 300, 3))
# Evaluation pre-processing: resize to SSD net shape.
image_pre, labels_pre, bboxes_pre, bbox_img = ssd_vgg_preprocessing.preprocess_for_eval(
    img_input, None, None, net_shape, data_format, resize=ssd_vgg_preprocessing.Resize.WARP_RESIZE)
image_4d = tf.expand_dims(image_pre, 0)

# Define the SSD model.
reuse = True if 'ssd_net' in locals() else None
ssd_net = ssd_vgg_300.SSDNet()
with slim.arg_scope(ssd_net.arg_scope(data_format=data_format)):
    predictions, localisations, _, _ = ssd_net.net(image_4d, is_training=False, reuse=reuse)

# Restore SSD model.
ckpt_filename = '../checkpoints/ssd_300_vgg.ckpt'
# ckpt_filename = '../checkpoints/VGG_VOC0712_SSD_300x300_ft_iter_120000.ckpt'
isess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(isess, ckpt_filename)

# SSD default anchor boxes.
ssd_anchors = ssd_net.anchors(net_shape)

INFO:tensorflow:Restoring parameters from ../checkpoints/ssd_300_vgg.ckpt


## Post-processing pipeline

The SSD outputs need to be post-processed to provide proper detections. Namely, we follow these common steps:

* Select boxes above a classification threshold;
* Clip boxes to the image shape;
* Apply the Non-Maximum-Selection algorithm: fuse together boxes whose Jaccard score > threshold;
* If necessary, resize bounding boxes to original image shape.

In [24]:
# Main image processing routine.
def process_image(img, select_threshold=0.5, nms_threshold=.45, net_shape=(300, 300)):
    # Run SSD network.
    start = time.time()
    # isess=tf.InteractiveSession(config=config)
    rimg, rpredictions, rlocalisations, rbbox_img = isess.run([image_4d, predictions, localisations, bbox_img],
                                                              feed_dict={img_input: img})
    print "isess.run耗时:",time.time()-start
    
    # Get classes and bboxes from the net outputs.
    rclasses, rscores, rbboxes = np_methods.ssd_bboxes_select(
            rpredictions, rlocalisations, ssd_anchors,
            select_threshold=select_threshold, img_shape=net_shape, num_classes=21, decode=True)
    
    rbboxes = np_methods.bboxes_clip(rbbox_img, rbboxes)
    rclasses, rscores, rbboxes = np_methods.bboxes_sort(rclasses, rscores, rbboxes, top_k=400)
    rclasses, rscores, rbboxes = np_methods.bboxes_nms(rclasses, rscores, rbboxes, nms_threshold=nms_threshold)
    # Resize bboxes to original image shape. Note: useless for Resize.WARP!
    rbboxes = np_methods.bboxes_resize(rbbox_img, rbboxes)
    return rclasses, rscores, rbboxes

In [8]:
# Test on some demo image and visualize output.
path = '../demo/'
image_names = sorted(os.listdir(path))

# init.run()

# for image in image_names:
#     img = mpimg.imread(path + image)
    
#     rclasses, rscores, rbboxes =  process_image(img)

#     # visualization.bboxes_draw_on_img(img, rclasses, rscores, rbboxes, visualization.colors_plasma)

#     #    if 15 in rclasses:
#     visualization.plt_bboxes(img, rclasses, rscores, rbboxes)

In [9]:
IMAGE_SIZE_X = 300
IMAGE_SIZE_Y = 300

In [20]:
def placeholder_inputs(batch_size):
    images_placeholder = tf.placeholder(tf.uint8, shape=(batch_size, 300, 300, 3))
    return images_placeholder

In [21]:
def fill_feed_dict(images_feed, images_pl):
    feed_dict = {
      images_pl: images_feed
    }
    return feed_dict

In [22]:
test_images = []
path = '../demo/'

for filename in ['img220.jpg', 'img221.jpg', 'img222.jpg', 'img223.jpg']:
#     image = Image.open(path+filename)
#     image = image.resize((IMAGE_SIZE_X,IMAGE_SIZE_Y))
    
    image = mpimg.imread(path + filename)
    image = image.resize((IMAGE_SIZE_X,IMAGE_SIZE_Y))
#     print np.array(image).dtype
    test_images.append(image)

    
test_images = np.array(test_images)

##test_images = test_images.reshape(4,IMAGE_SIZE_X*IMAGE_SIZE_Y*3)
print test_images.shape

images_placeholder=placeholder_inputs(4)

print images_placeholder.shape

feed_dict = fill_feed_dict(test_images,images_placeholder)
isess.run([image_4d, predictions, localisations, bbox_img], feed_dict=feed_dict)

(4,)
(4, 300, 300, 3)


TypeError: long() argument must be a string or a number, not 'NoneType'